<a href="https://colab.research.google.com/github/VijayaVajjhala/Product-Recommender-Agent/blob/main/Product_Recommendor_and_Comparator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install neo4j

  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import json
import hashlib # For generating review_id
from datetime import datetime

In [9]:
NEO4J_URI = "neo4j+s://49db1554.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "em3IQa3dY6_A5fHCg21eRDd138qX0Xxms0xyljCBbQ8"
NEO4J_DATABASE = "neo4j" # Default database name for AuraDB

In [10]:
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print("Neo4j connection successful!")
except Exception as e:
    print(f"Neo4j connection failed: {e}")
    print("Please ensure your NEO4J_URI, NEO4J_USERNAME, and NEO4J_PASSWORD are correct.")
    print("If you are running locally, ensure your Neo4j instance is running and accessible.")
    raise

Neo4j connection successful!


In [11]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding model loaded.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.


In [12]:
from transformers import pipeline

In [31]:
print("Loading sentiment analysis model...")
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
print("Model loaded.")

Loading sentiment analysis model...


Device set to use cpu


Model loaded.


In [32]:
def perform_sentiment_analysis(texts):
    """
    Performs sentiment analysis on a list of texts using a pre-trained
    Hugging Face sentiment analysis model.

    Args:
        texts (list of str): A list of text strings to analyze.

    Returns:
        list of dict: A list of dictionaries, where each dictionary
                      contains the 'label' (e.g., 'POSITIVE', 'NEGATIVE')
                      and 'score' for the sentiment of the corresponding text.
    """
    # Initialize the sentiment-analysis pipeline.
    # The 'distilbert-base-uncased-finetuned-sst-2-english' model is a
    # popular and efficient choice for binary (positive/negative) sentiment.
    # It's fine-tuned on the Stanford Sentiment Treebank (SST-2) dataset.


    # Perform sentiment analysis
    results = classifier(texts)

    return results


In [20]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
product_metadata_path = '/content/drive/MyDrive/Product Agent/meta_All_Beauty.jsonl.gz'
review_data_path = '/content/drive/MyDrive/Product Agent/All_Beauty.jsonl.gz'

Mounted at /content/drive


In [21]:
def read_jsonl_gz_to_pandas(file_path):
    """
    Reads a .jsonl.gz file into a pandas DataFrame.

    Args:
        file_path (str): The path to the .jsonl.gz file.

    Returns:
        pd.DataFrame: A DataFrame containing the data from the file.
                      Returns None if the file cannot be read.
    """
    try:
        # pd.read_json can directly handle gzipped files and JSON Lines format.
        # 'compression="gzip"' tells pandas to decompress the file.
        # 'lines=True' tells pandas that each line is a separate JSON object.
        df = pd.read_json(file_path, compression='gzip', lines=True)
        print(f"Successfully read {len(df)} records from {file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

In [22]:
products_df = read_jsonl_gz_to_pandas(product_metadata_path)

Successfully read 112590 records from /content/drive/MyDrive/Product Agent/meta_All_Beauty.jsonl.gz


In [23]:
reviews_df = read_jsonl_gz_to_pandas(review_data_path)

Successfully read 701528 records from /content/drive/MyDrive/Product Agent/All_Beauty.jsonl.gz


In [25]:
products_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,NaN
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,NaN
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,NaN
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,NaN
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,NaN


In [26]:
def preprocess_product(product_row):
    product_dict = product_row.to_dict()

    # Join list descriptions for embedding, keep original for LLM
    product_dict['description_joined'] = " ".join(product_dict['description']) if isinstance(product_dict['description'], list) else ""

    # Extract brand from 'details' dictionary
    product_dict['brand_name'] = product_dict['details'].get('Brand', 'Unknown')

    # Convert lists of dicts to JSON strings for easier storage if needed, or keep as lists
    # For now, keep them as lists as Neo4j can handle List<Map>
    product_dict['images_json'] = json.dumps(product_dict['images']) if isinstance(product_dict['images'], list) else "[]"
    product_dict['videos_json'] = json.dumps(product_dict['videos']) if isinstance(product_dict['videos'], list) else "[]"

    # Generate embedding for Product
    product_text_for_embedding = f"{product_dict['title']} {product_dict['description_joined']} {product_dict['main_category']} {product_dict['brand_name']}"
    product_dict['embedding'] = embedding_model.encode(product_text_for_embedding).tolist()

    return product_dict

In [27]:
products_processed_df = products_df.apply(preprocess_product, axis=1)

In [37]:
def preprocess_review(review_row):
    """
    Preprocesses a review row, adding a unique ID, converting timestamp,
    performing real sentiment analysis, and generating an embedding.

    Args:
        review_row (pandas.Series): A single row from a DataFrame containing
                                     review data. Expected columns:
                                     'user_id', 'parent_asin', 'asin',
                                     'timestamp', 'text', 'title', 'images', 'rating'.

    Returns:
        dict: A dictionary containing the processed review data,
              including 'review_id', 'reviewTime', 'sentiment_label',
              'sentiment_probability', 'images_json', and 'embedding'.
    """
    review_dict = review_row.to_dict()

    # Generate a unique review_id using a hash for robust identification
    unique_string = f"{review_dict['user_id']}-{review_dict['parent_asin']}-{review_dict['asin']}-{review_dict['timestamp']}-{review_dict['text']}"
    review_dict['review_id'] = hashlib.sha256(unique_string.encode()).hexdigest()

    # Convert timestamp (Unix epoch) to ISO 8601 format for consistency
    # Ensure 'timestamp' exists and is an integer/float
    if 'timestamp' in review_dict and isinstance(review_dict['timestamp'], (int, float)):
        # Convert to datetime and then to ISO format string
        review_dict['reviewTime'] = datetime.fromtimestamp(review_dict['timestamp']).isoformat()
    else:
        review_dict['reviewTime'] = None # Or handle error as appropriate

    # --- Replace mock sentiment analysis with real Hugging Face model ---
    # The perform_sentiment_analysis function expects a list of texts,
    # so we pass the current review's text as a single-element list.
    # The result will be a list containing one dictionary.
    if 'text' in review_dict and review_dict['text']: # Ensure text is not empty
        # Truncate text to fit within model's maximum sequence length (typically 512 for BERT-like models)
        # A simple approach is to take the first N characters, but tokenization is more accurate.
        # For simplicity and robustness against variable token lengths, we'll truncate characters.
        # A safer approach might involve tokenizing and truncating tokens, but this requires
        # access to the model's tokenizer which might not be readily available here without
        # modifying the perform_sentiment_analysis function or passing the tokenizer.
        # Let's assume a rough character limit based on the token limit. 512 tokens is roughly ~400-500 words,
        # which is significantly more than 512 characters. Truncating to 512 characters is a safe bet
        # to stay within token limits for most cases, though not perfect.
        max_chars = 500 # A conservative character limit to likely stay within 512 tokens
        truncated_text = review_dict['text'][:max_chars]

        sentiment_results = perform_sentiment_analysis([truncated_text])
        # Extract the label and score from the first (and only) result
        review_dict['sentiment_label'] = sentiment_results[0]['label']
        review_dict['sentiment_probability'] = sentiment_results[0]['score']
    else:
        review_dict['sentiment_label'] = "neutral" # Default for empty text
        review_dict['sentiment_probability'] = 0.5


    # Convert images list of dicts to JSON string for storage, if not handled directly by Neo4j driver
    # This ensures that if 'images' is a list (as expected for JSON objects), it's converted.
    # Otherwise, it defaults to an empty JSON array string.
    review_dict['images_json'] = json.dumps(review_dict.get('images', [])) if isinstance(review_dict.get('images'), list) else "[]"

    # Generate embedding for Review using the external 'embedding_model'
    # It's good practice to combine relevant text fields for a comprehensive embedding.
    review_text_for_embedding = f"{review_dict.get('title', '')} {review_dict.get('text', '')}"
    # Ensure embedding_model is initialized before this function is called
    if 'embedding_model' in globals() and embedding_model:
        review_dict['embedding'] = embedding_model.encode(review_text_for_embedding).tolist()
    else:
        review_dict['embedding'] = [] # Default to empty list if embedding_model not available
        print("Warning: 'embedding_model' not found or initialized. Embedding generation skipped.")


    return review_dict

In [ ]:
reviews_processed_df = reviews_df.apply(preprocess_review, axis=1)

In [ ]:
def run_cypher_query(query, parameters=None):
    with driver.session(database=NEO4J_DATABASE) as session:
        result = session.run(query, parameters)
        return [record for record in result]

In [ ]:
print("\n--- Creating Constraints ---")
constraints = [
    "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Product) REQUIRE p.parent_asin IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.user_id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (r:Review) REQUIRE r.review_id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Category) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (b:Brand) REQUIRE b.name IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (f:Feature) REQUIRE f.name IS UNIQUE"
]

In [ ]:
for constraint in constraints:
    try:
        run_cypher_query(constraint)
        print(f"  - {constraint.split('REQUIRE ')[1].split(' IS UNIQUE')[0]} constraint created/ensured.")
    except Exception as e:
        print(f"  - Failed to create constraint '{constraint}': {e}")

In [ ]:
# Batch size for ingestion
BATCH_SIZE = 1000

In [ ]:
print("\n--- Creating Nodes ---")

In [ ]:
# Create Product nodes
print("  - Creating Product nodes...")
product_data_for_cypher = products_processed_df[[
    'parent_asin', 'title', 'main_category', 'average_rating', 'rating_number',
    'description', 'price', 'images_json', 'videos_json', 'store', 'brand_name', 'embedding'
]].to_dict(orient='records')

for i in range(0, len(product_data_for_cypher), BATCH_SIZE):
    batch = product_data_for_cypher[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS product
    MERGE (p:Product {parent_asin: product.parent_asin})
    SET p.title = product.title,
        p.main_category = product.main_category,
        p.average_rating = product.average_rating,
        p.rating_number = product.rating_number,
        p.description = product.description,
        p.price = product.price,
        p.images = apoc.convert.fromJsonList(product.images_json),
        p.videos = apoc.convert.fromJsonList(product.videos_json),
        p.store = product.store,
        p.embedding = product.embedding
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(product_data_for_cypher)} Product nodes.")

In [ ]:
# Create User nodes
print("  - Creating User nodes...")
user_data_for_cypher = reviews_processed_df[['user_id']].drop_duplicates().to_dict(orient='records')
for i in range(0, len(user_data_for_cypher), BATCH_SIZE):
    batch = user_data_for_cypher[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS user
    MERGE (u:User {user_id: user.user_id})
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(user_data_for_cypher)} User nodes.")

In [ ]:
# Create Review nodes
print("  - Creating Review nodes...")
review_data_for_cypher = reviews_processed_df[[
    'review_id', 'asin', 'rating', 'title', 'text', 'images_json',
    'timestamp', 'reviewTime', 'verified_purchase', 'helpful_vote',
    'sentiment', 'embedding', 'user_id', 'parent_asin' # Include IDs for relationship creation later
]].to_dict(orient='records')

In [ ]:
for i in range(0, len(review_data_for_cypher), BATCH_SIZE):
    batch = review_data_for_cypher[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS review
    MERGE (r:Review {review_id: review.review_id})
    SET r.asin = review.asin,
        r.rating = review.rating,
        r.title = review.title,
        r.text = review.text,
        r.images = apoc.convert.fromJsonList(review.images_json),
        r.timestamp = review.timestamp,
        r.reviewTime = review.reviewTime,
        r.verified_purchase = review.verified_purchase,
        r.helpful_vote = review.helpful_vote,
        r.sentiment = review.sentiment,
        r.embedding = review.embedding
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(review_data_for_cypher)} Review nodes.")

In [ ]:
# Create Category nodes and IS_A relationships (Hierarchy)
print("  - Creating Category nodes and hierarchy...")
all_categories = set()
for _, row in products_processed_df.iterrows():
    if isinstance(row['categories'], list):
        for category_path in row['categories']:
            all_categories.add(category_path)

In [ ]:
# Create unique category nodes first
unique_category_names = [{"name": cat} for cat in all_categories]
for i in range(0, len(unique_category_names), BATCH_SIZE):
    batch = unique_category_names[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS category
    MERGE (c:Category {name: category.name})
    SET c.embedding = $embedding_model.encode(category.name).tolist()
    """
    run_cypher_query(query, parameters={"batch": batch, "embedding_model": embedding_model})
print(f"    Created {len(unique_category_names)} unique Category nodes.")

In [ ]:
# Create IS_A relationships for category hierarchy
category_hierarchy_relationships = []
for _, row in products_processed_df.iterrows():
    if isinstance(row['categories'], list) and len(row['categories']) > 1:
        for i in range(len(row['categories']) - 1):
            child_cat = row['categories'][i]
            parent_cat = row['categories'][i+1]
            category_hierarchy_relationships.append({"child": child_cat, "parent": parent_cat})

In [ ]:
for i in range(0, len(category_hierarchy_relationships), BATCH_SIZE):
    batch = category_hierarchy_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (child:Category {name: rel.child})
    MATCH (parent:Category {name: rel.parent})
    MERGE (child)-[:IS_A]->(parent)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(category_hierarchy_relationships)} Category hierarchy relationships (IS_A).")

In [ ]:
# Create Brand nodes
print("  - Creating Brand nodes...")
unique_brands = products_processed_df['brand_name'].dropna().unique()
brand_data_for_cypher = [{"name": brand} for brand in unique_brands]
for i in range(0, len(brand_data_for_cypher), BATCH_SIZE):
    batch = brand_data_for_cypher[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS brand
    MERGE (b:Brand {name: brand.name})
    SET b.embedding = $embedding_model.encode(brand.name).tolist()
    """
    run_cypher_query(query, parameters={"batch": batch, "embedding_model": embedding_model})
print(f"    Created {len(unique_brands)} Brand nodes.")

In [ ]:
# Create Feature nodes
print("  - Creating Feature nodes...")
all_features = set()
for features_list in products_processed_df['features'].dropna():
    if isinstance(features_list, list):
        for feature in features_list:
            all_features.add(feature)

feature_data_for_cypher = [{"name": feature} for feature in all_features]
for i in range(0, len(feature_data_for_cypher), BATCH_SIZE):
    batch = feature_data_for_cypher[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS feature
    MERGE (f:Feature {name: feature.name})
    SET f.embedding = $embedding_model.encode(feature.name).tolist()
    """
    run_cypher_query(query, parameters={"batch": batch, "embedding_model": embedding_model})
print(f"    Created {len(all_features)} Feature nodes.")

In [ ]:
# --- 4.3. Create Relationships ---

print("\n--- Creating Relationships ---")


In [ ]:
# WROTE relationships (User -> Review)
print("  - Creating WROTE relationships...")
wrote_relationships = reviews_processed_df[['user_id', 'review_id']].to_dict(orient='records')
for i in range(0, len(wrote_relationships), BATCH_SIZE):
    batch = wrote_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (u:User {user_id: rel.user_id})
    MATCH (r:Review {review_id: rel.review_id})
    MERGE (u)-[:WROTE]->(r)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(wrote_relationships)} WROTE relationships.")

In [ ]:
# REVIEWS relationships (Review -> Product)
print("  - Creating REVIEWS relationships...")
reviews_relationships = reviews_processed_df[['review_id', 'parent_asin', 'asin']].to_dict(orient='records')
for i in range(0, len(reviews_relationships), BATCH_SIZE):
    batch = reviews_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (r:Review {review_id: rel.review_id})
    MATCH (p:Product {parent_asin: rel.parent_asin})
    MERGE (r)-[:REVIEWS {asin: rel.asin}]->(p)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(reviews_relationships)} REVIEWS relationships.")

In [ ]:
# HAS_CATEGORY relationships (Product -> Category)
print("  - Creating HAS_CATEGORY relationships...")
has_category_relationships = []
for _, row in products_processed_df.iterrows():
    parent_asin = row['parent_asin']
    if isinstance(row['categories'], list):
        for category_name in row['categories']:
            has_category_relationships.append({"parent_asin": parent_asin, "category_name": category_name})

for i in range(0, len(has_category_relationships), BATCH_SIZE):
    batch = has_category_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (p:Product {parent_asin: rel.parent_asin})
    MATCH (c:Category {name: rel.category_name})
    MERGE (p)-[:HAS_CATEGORY]->(c)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(has_category_relationships)} HAS_CATEGORY relationships.")

In [ ]:
# HAS_BRAND relationships (Product -> Brand)
print("  - Creating HAS_BRAND relationships...")
has_brand_relationships = products_processed_df[['parent_asin', 'brand_name']].dropna().to_dict(orient='records')
for i in range(0, len(has_brand_relationships), BATCH_SIZE):
    batch = has_brand_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (p:Product {parent_asin: rel.parent_asin})
    MATCH (b:Brand {name: rel.brand_name})
    MERGE (p)-[:HAS_BRAND]->(b)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(has_brand_relationships)} HAS_BRAND relationships.")

In [ ]:
# HAS_FEATURE relationships (Product -> Feature)
print("  - Creating HAS_FEATURE relationships...")
has_feature_relationships = []
for _, row in products_processed_df.iterrows():
    parent_asin = row['parent_asin']
    if isinstance(row['features'], list):
        for feature_name in row['features']:
            has_feature_relationships.append({"parent_asin": parent_asin, "feature_name": feature_name})

for i in range(0, len(has_feature_relationships), BATCH_SIZE):
    batch = has_feature_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (p:Product {parent_asin: rel.parent_asin})
    MATCH (f:Feature {name: rel.feature_name})
    MERGE (p)-[:HAS_FEATURE]->(f)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(has_feature_relationships)} HAS_FEATURE relationships.")

In [ ]:
# BOUGHT_TOGETHER_WITH relationships (Product -> Product)
print("  - Creating BOUGHT_TOGETHER_WITH relationships...")
bought_together_relationships = []
for _, row in products_processed_df.iterrows():
    source_parent_asin = row['parent_asin']
    if isinstance(row['bought_together'], list):
        for target_parent_asin in row['bought_together']:
            # Ensure the target product exists in our dataset
            if target_parent_asin in products_processed_df['parent_asin'].values:
                bought_together_relationships.append({
                    "source_parent_asin": source_parent_asin,
                    "target_parent_asin": target_parent_asin
                })

In [ ]:
# Deduplicate for MERGE
unique_bought_together = set()
for rel in bought_together_relationships:
    # Ensure a symmetric relationship is created only once if (A)-[:REL]->(B) and (B)-[:REL]->(A) are equivalent
    # Or enforce direction based on data
    pair = tuple(sorted((rel["source_parent_asin"], rel["target_parent_asin"])))
    if pair not in unique_bought_together:
        bought_together_relationships.append(rel)
        unique_bought_together.add(pair)

for i in range(0, len(bought_together_relationships), BATCH_SIZE):
    batch = bought_together_relationships[i:i + BATCH_SIZE]
    query = """
    UNWIND $batch AS rel
    MATCH (p1:Product {parent_asin: rel.source_parent_asin})
    MATCH (p2:Product {parent_asin: rel.target_parent_asin})
    MERGE (p1)-[:BOUGHT_TOGETHER_WITH]->(p2)
    """
    run_cypher_query(query, parameters={"batch": batch})
print(f"    Created {len(bought_together_relationships)} BOUGHT_TOGETHER_WITH relationships.")

In [ ]:
print("\n--- Creating Vector Indexes ---")

vector_index_queries = [
    {"name": "product_embeddings", "labels": "Product", "property": "embedding", "dimensions": embedding_model.get_sentence_embedding_dimension()},
    {"name": "review_embeddings", "labels": "Review", "property": "embedding", "dimensions": embedding_model.get_sentence_embedding_dimension()},
    {"name": "category_embeddings", "labels": "Category", "property": "embedding", "dimensions": embedding_model.get_sentence_embedding_dimension()},
    {"name": "brand_embeddings", "labels": "Brand", "property": "embedding", "dimensions": embedding_model.get_sentence_embedding_dimension()},
    {"name": "feature_embeddings", "labels": "Feature", "property": "embedding", "dimensions": embedding_model.get_sentence_embedding_dimension()},
]

In [ ]:
for idx in vector_index_queries:
    try:
        # Check if index exists before creating (Neo4j does this automatically with IF NOT EXISTS)
        query = (
            f"CREATE VECTOR INDEX {idx['name']} IF NOT EXISTS "
            f"FOR (n:{idx['labels']}) ON (n.{idx['property']}) "
            f"OPTIONS {{nodeLabels:['{idx['labels']}'], propertyName:'{idx['property']}', "
            f"embeddingDimensions:{idx['dimensions']}, similarityFunction:'cosine'}}"
        )
        run_cypher_query(query)
        print(f"  - Vector Index '{idx['name']}' created/ensured on :{idx['labels']}.{idx['property']}")
    except Exception as e:
        print(f"  - Failed to create vector index '{idx['name']}': {e}")
        print("    Ensure your Neo4j version is 5.16 or higher for Node Property Vector Indexes.")

In [ ]:
# Close the driver connection
driver.close()
print("\nNeo4j driver closed. Graph creation complete!")

In [ ]:
print("\n--- Next Steps: Integrating with LangChain for Graph RAG ---")
print("Once the graph is populated, you can use LangChain's Neo4jGraph to interact with it.")
print("Example setup for LangChain:")

langchain_example = """
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI # Or any other LLM

In [ ]:
# Initialize Neo4jGraph
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)